In [1]:
from dotenv import load_dotenv
from IPython.display import display, Markdown
load_dotenv()

True

In [2]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
import boto3
from langchain_aws import ChatBedrock
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain.agents import create_agent # <--- Importante

# 1. Configurar el modelo (Asegúrate de definir 'model_id')


llm = ChatBedrock(
    model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 0.9,
    }
)

## Local MCP server

In [4]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [5]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [29]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=prompt
)


In [30]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [31]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='2c022b79-bc0a-470a-845e-4064e3f410de'),
              AIMessage(content='\n\nsearch_web "langchain-mcp-adapters library"', additional_kwargs={'usage': {'prompt_tokens': 160, 'completion_tokens': 15, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 175}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 160, 'completion_tokens': 15, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 175}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0', 'model_provider': 'bedrock', 'model_name': 'us.meta.llama4-maverick-17b-instruct-v1:0'}, id='lc_run--019b4b5c-b3d3-71e3-86f6-35426594862b-0', usage_metadata={'input_tokens': 160, 'output_tokens': 15, 'total_tokens': 175, 'input_tok

## Online MCP

In [35]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
    
)

tools = await client.get_tools()

In [38]:
agent = create_agent(
    model="amazon.nova-lite-v1:0",
    tools=tools,
)

In [39]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='dfd73f48-8110-4a25-9e52-78b728d5c9b7'),
              AIMessage(content=[{'type': 'text', 'text': "<thinking>The User has asked for the current time, but did not specify a timezone. I will use the 'get_current_time' tool with the default timezone 'America/New_York' to get the current time.</thinking>\n"}, {'type': 'tool_use', 'name': 'get_current_time', 'input': {'timezone': 'America/New_York'}, 'id': 'tooluse_z7bI-olDS-SBAb_SQNyKWw'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '4c792eaa-ff28-4d6e-adb0-b2729602e8f8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 23 Dec 2025 13:21:02 GMT', 'content-type': 'application/json', 'content-length': '533', 'connection': 'keep-alive', 'x-amzn-requestid': '4c792eaa-ff28-4d6e-adb0-b2729602e8f8'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [780]}, 'model_provider': 'bedrock_conv